# Water Quality Network
For our network project, Cassie and I are thinking of creating a bipartite network where the nodes are pollutants and the links are water facilities whose measurements of those pollutants are above the threshold. By doing this linking, we hope to see what the biggest pollutants are, how pollutants might connect to one another, and if there are any pollutants that we should be worried about.

## Data Sources
We got our data from USGS (U.S. Geological Survey). I'll need to get the exact link later.

### MUST PIP INSTALL
* numpy
* pandas
* matplotlib
* xlrd

In [1]:
import re
import time
import numpy as np
import pandas as pd
import networkx as nx
from io import StringIO
import matplotlib.pyplot as plt
from networkx.algorithms import bipartite
%matplotlib inline

## Looking at the Quality data
Since I'm not too familiar with the data, let's load it in and take a look. There's some preliminary things that I do know, which will be come prevelant when you look at the code. For instance, you can break up the code into chunks by splitting on instances of "#\n". The # comes from the file header and most "sections" within the header are separated by # followed by a new line. The last item of the split is the actual data.

Let's start by loading in everything!

In [2]:
def load_quality_data(file_name):
    quality_data = []
    with open(file_name, 'r') as file:
        quality_data = file.read().split("#\n")

    if not quality_data:
        print("File {} was unable to be read.".format(file_name))
    return quality_data

In [3]:
file_name = "LA_Water_Quality_Data.txt"
quality_data = load_quality_data(file_name)
print("Number of sections:", len(quality_data))
print("2nd section:", quality_data[2], sep = "\n")
print("Number of characters in actual data:", len(quality_data[15]))

Number of sections: 16
2nd section:
# U.S. Geological Survey
# 
# This file contains selected water-quality data for stations in the National Water Information 
# System water-quality database.  Explanation of codes found in this file are followed by
# the retrieved data.

Number of characters in actual data: 9736982


Great! We have 15 chunks of text when doing that split (which feels a bit better than doing it line-by-line). Here's a breakdown of what's inside:
    #  0:                               #  8: coll_ent_cd  
    #  1: File created...               #  9: medium_cd  
    #  2: U.S. Geological Survey        # 10: tu_id  
    #  3: The data you have...          # 11: body_part_id  
    #  4: To view additional...         # 12: remark_cd  
    #  5: Param_id      - parameter     # 13: Data for the following sites...  
    #  6: sample_start_time_datum_cd    # 14: WARNING: some spreadsheet...  
    #  7: tm_datum_rlbty_cd             # 15: Data!  

I've already glanced at the file in Excel and figured out how to parse all of the parameters and their descriptions, which will probably be useful later on. 

### Parameters
There are about a thousand parameters, most of them are in the format of a p + 5-digit number (i.e. `p62168`). Not very descriptive, but the 6th entry of our data header has the actual descriptions for each parameter. These descriptions have a lot of information, such as the pollutant name, filtered vs unfiltered, and units of measurement.

We want to later extract the pollutant and if the sample was filtered or unfiltered. For now, let's just make a dictionary to get the description for the parameters.

`get_params_def`: function
* Input is `param_header`, which is a giant string with `"\n"` characters separating each line
* Each "line" has the `p#####` parameter label and the description, 

In [4]:
# inputs the part of the header that contains the parameter label followed by its meaning
# outputs a dictionary where the key is the label (lower case) and the value is the description
def get_params_def(param_header):
    params_def_dict = {} # Key is p#####, value is description, which contains the pollutant
    params = param_header.split("\n")
    params_pattern = re.compile("# +(\w+) +- +(.+)")

    for param in params:
        a = params_pattern.search(param)

        if a: 
            altered_description = re.sub("  +", ", ", 
                                         a.group(2).replace("filtered (", "filtered, ("))
            params_def_dict[a.group(1).lower()] = altered_description
            #print(a.group(1) + ":", a.group(2).split(',')[-1])

    return params_def_dict

In [5]:
params_def_dict = get_params_def(quality_data[5])
print("Total number of parameters measured:", len(params_def_dict))
ex_param_dict = list(params_def_dict.keys())[len(params_def_dict.keys())//2]
print("Example:\n\tkey   = {}\n\tvalue = {}".format(ex_param_dict, params_def_dict[ex_param_dict]))

Total number of parameters measured: 1046
Example:
	key   = p62168
	value = Fipronil sulfone, water, filtered, recoverable, micrograms per liter


We have the list of all the parameters, but we're really only interested in a few of the metadata ones and we'll need to go through the measurement ones to determine which ones we want.

Since we know the metadata parameters we want, we can put them into `needed_params`:
* the site number, `site_no`
* the date the measurement was taken `sample_dt`
* the time the measurement was taken `sample_tm`

Since we want to go through specifically the measurements (p + 5-digit params), we'll create a list of all those named `data_params` that we'll use later to subset the data.

In [6]:
# Get only the parameters we're interested in, which are the site info, date/times of sampling,
#  and all of the measurements (the p + 5-digit params)
needed_params = ["site_no", "sample_dt", "sample_tm"]
data_params = [param for param in params_def_dict if param[0] == "p"]

### Quality Data 
Now that we have the parameters, we can grab the actual measurements from `quality_data`. The data is a tab and newline-separated chunk of text where the tabs separate parameter measurements and the newlines separate measurements of a specific time. The data also has an extra row underneath the header that doesn't seem of any use to us, so we can disregard it.

In [7]:
test = quality_data[15]

In [8]:
# Time: 0.16467714309692383
start_time = time.time()
subbed_test = re.sub("\t[AERMUV]", 
              "\t", test.replace("USGS", 
                                 "~~~~").replace(" ", 
                                                 "").replace("<", 
                                                             "").replace(">",
                                                                         "")).replace("~~~~", 
                                                                                      "USGS")
print(time.time() - start_time)

0.2025909423828125


In [11]:
types_per_param = {param:"float" if param[0]=="p" else "str" for param in params_def_dict}

In [12]:
data_to_use = pd.read_csv(StringIO(subbed_test), sep='\t', 
                          dtype=types_per_param, header=0, skiprows=[1])
# I find this a bit concerning...: https://stackoverflow.com/questions/24251219/pandas-read-csv-low-memory-and-dtype-options
# I'd want to specify type, but it seems like there's a lot of strings within supposedly numerical columns...
print(data_to_use.shape)
display(data_to_use.head(2))

(7763, 1046)


,agency_cd,site_no,sample_dt,sample_tm,sample_end_dt,sample_end_tm,sample_start_time_datum_cd,tm_datum_rlbty_cd,coll_ent_cd,medium_cd,...,p99856,p99871,p99931,p99947,p99958,p99959,p99963,p99972,p99994,p99995
0,USGS,332031118504001,2000-10-24,14:30,NaN,NaN,PDT,T,USGS-WRD,WG,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USGS,333420118060501,2000-11-09,09:30,NaN,NaN,PST,T,USGS-WRD,WG,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now that we've loaded in all the data successfully, let's get the subset of the data that only contains measurements. Calling it `data_to_use_numbers`.

In [13]:
data_to_use_numbers = data_to_use[data_params]
data_to_use_numbers.head()

,p00003,p00004,p00005,p00008,p00009,p00010,p00011,p00020,p00021,p00025,...,p99856,p99871,p99931,p99947,p99958,p99959,p99963,p99972,p99994,p99995
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,18.5,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,973.0,90.7,99.8
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
#string = data_to_use_numbers.to_string()

In [15]:
#string.replace("NaN", "~~~").replace("\t", "===============")

In [16]:
#data_to_use.columns.values.tolist()

In [17]:
#data_to_use_numbers = data_to_use_numbers.replace("[<> A-Za-z]", "", regex=True)
#data_to_use_numbers = data_to_use_numbers.replace(" ", "", regex=True)
#data_to_use_numbers = data_to_use_numbers.replace(r'^$', np.nan, regex=True)

In [18]:
#data_to_use_numbers = data_to_use_numbers.astype(np.float64)

In [19]:
all(data_to_use_numbers.dtypes == "float64")

True

In [20]:
#total_data = pd.concat([data_to_use[needed_params], data_to_use_numbers], axis=1)

In [21]:
#total_data.head()

## Splitting params into filtered and unfiltered
* unfiltered = grab the ground water as is - you have extra sediment somehow
* filtered = you filter out the ground water 
* do analysis with filtered and unfiltered, but DON'T MIX
* higher pollution for unfiltered properties

#### Note
There's some descrepencies in these parameters. We'll deal with this later.

In [22]:
def filter_params_by(params_def_dict, by="filtered", opposite=False):
    filtered_params = {}
    for param_def in params_def_dict.items():
        component_def = param_def[1].split(", ")
        component = component_def[0].lower()
        
        if (opposite and by not in component_def) or (not opposite and by in component_def):
            if component not in filtered_params:
                filtered_params[component] = []
            filtered_params[component].append(param_def[0])
            
    return filtered_params

In [23]:
#said_filtered_pollutants = filter_params_by(params_def_dict, by="filtered", opposite=False)
all_filtered_params = filter_params_by(params_def_dict, by="unfiltered", opposite=True)

## Next Step: Getting the pollutants 

### Try the CA Pollutants instead 

In [32]:
ca_pollutant_file = pd.ExcelFile("CA_thresholds_compliation_filtered.xlsx")
ca_pollutant_info = ca_pollutant_file.parse()

In [33]:
ca_pollutant_info.head()

,Name1,Organic_Inorganic,CA_Prim_MCL,CA_Prim_MCL_2,CA_Prim_MCL_unit,CA_Prim_MCL_date,CA_Sec_MCL,CA_Sec_MCL_2,CA_Sec_MCL_unit,CA_Sec_MCL_date,...,CA_BayEst_Health_unit,CA_BayEst_Health_date,NAWQC_Health_WF,NAWQC_Health_WF_2,NAWQC_Health_WF_unit,NAWQC_Health_WF_date,NAWQC_Health_F,NAWQC_Health_F_2,NAWQC_Health_F_unit,NAWQC_Health_F_date
0,Acenaphthene,Organic,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,...,NaN,2000-05-18,70.0,NaN,NaN,2015-06-29,90.0,NaN,NaN,2015-06-29
1,Acenaphthylene,Organic,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,...,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT
2,Acetochlor,Organic,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,...,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT
3,Acetone,Organic,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,...,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT
4,Acetonitrile,Organic,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,...,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT


In [34]:
ca_pollutant_info.iloc[:, :2] = ca_pollutant_info.iloc[:, :2].replace(to_replace=np.nan, value="")
#names = set(ca_pollutant_info["Name1"].str.lower().tolist()) # gives us all the names we're working with
ca_pollutant_info["Name1"] = ca_pollutant_info["Name1"].str.lower()
names = ca_pollutant_info["Name1"]


In [73]:
count = 0
names = set(ca_pollutant_info["Name1"].tolist())
all_params = set(list(all_filtered_params.keys()))
rows_containing_pollutant = [] # will be used to grab the rows from ca_pollutant_info
names_of_pollutant = []        # 
common_pollutants_p = []
reverse_params_def_dict = {}
for column in names.intersection(all_params):
    match_col = [i for i,name in enumerate(names) if name == column]
    #print(column, match_col, type(match_col))
    rows_containing_pollutant += match_col
    for i in range(len(match_col)):
        names_of_pollutant.append(column)
    for p_column in all_filtered_params[column]:
        reverse_params_def_dict[p_column] = column
        common_pollutants_p.append(p_column)
    count += 1
total_data = pd.concat([data_to_use[needed_params], data_to_use_numbers[common_pollutants_p]], axis=1)

In [78]:
chosen_ca_pollutant_info = ca_pollutant_info.iloc[rows_containing_pollutant, :]
chosen_ca_pollutant_info.shape

(143, 90)

In [79]:
data_cols = [column for column in chosen_ca_pollutant_info.columns.values.tolist()[7:] if "unit" not in column and "date" not in column and "Ag_Goals" not in column]#and "_2" not in column]
chosen_ca_pollutant_data = chosen_ca_pollutant_info[data_cols]
chosen_ca_pollutant_data.head()

,CA_Sec_MCL_2,USEPA_Prim_MCL,USEPA_Prim_MCL_2,USEPA_Sec_MCL,USEPA_Sec_MCL_2,USEPA_MCL_Goal,USEPA_MCL_Goal_2,CA_PHG,CA_PHG_2,CA_Action_Level,...,CA_Inland_Health_DW,CA_Inland_Health_DW_2,CA_Inland_Health_Other,CA_Inland_Health_Other_2,CA_BayEst_Health,CA_BayEst_Health_2,NAWQC_Health_WF,NAWQC_Health_WF_2,NAWQC_Health_F,NAWQC_Health_F_2
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
x = chosen_ca_pollutant_data.iloc[19,:]
x[(x.notna()) &  (x>0)]

USEPA_Prim_MCL            4.0
USEPA_MCL_Goal            4.0
CA_PHG                    1.0
USEPA_IRIS_RfD           14.0
USEPA_HA_NonCancer    30000.0
Name: 33, dtype: float64

In [81]:
def find_threshold_minimum(x):
    x_data = x[(x.notna()) &  (x > 0)]
    if len(x_data) > 0:
        return min(x_data)
    else:
        return 999999999
    
min_thresholds = chosen_ca_pollutant_data.apply(find_threshold_minimum, axis=1)
min_thresholds_cols = chosen_ca_pollutant_data.idxmin(axis=1)

In [82]:
for row in chosen_ca_pollutant_data.iterrows():
    x = row[1]
    x_data = x[(x.notna()) &  (x > 0)]
    if len(x_data) > 0:
        if max(x_data)/min(x_data) > 1000:
            print(chosen_ca_pollutant_info["Name1"][row[0]])
            print(min(x_data), max(x_data), )
            print("\n")

acifluorfen
1.0 2000.0


alachlor
0.4 700.0


arsenic
0.0014 10.0


beryllium
1.0 30000.0


bromodichloromethane
0.27 600.0


cadmium
0.0023 5.0


trichloromethane
0.26 2000.0


cyanide
4.0 220000.0


p,p'-ddd
0.00083 1.0


dibromochloromethane
0.4 18000.0


1,2-dibromoethane
0.01 63.0


1,1-dichloroethene
0.057 20000.0


trans-1,2-dichloroethene
60.0 140000.0


dichloromethane
0.002 5000.0


1,3-dichloropropene
0.2 1700.0


dimethyl phthalate
2000.0 2900000.0


2,4-dinitrophenol
10.0 14000.0


heptachlor
0.00021 10.0


heptachlor epoxide
0.0001 10.0


hexachlorobenzene
0.00075 50.0


1,1,2,2-tetrachloroethane
0.1 3000.0


toxaphene
0.00073 8.75


1,1,1-trichloroethane
17.0 200000.0


2,4,6-trichlorophenol
0.5 2500.0


vinyl chloride
0.02 3000.0




In [85]:
ca_pollutant_thresholds = pd.DataFrame({"Pollutant":names_of_pollutant, "Min Value":min_thresholds, "Threshold Adhered To":min_thresholds_cols})
ca_pollutant_thresholds = ca_pollutant_thresholds[ca_pollutant_thresholds["Threshold Adhered To"].notna()]
ca_pollutant_thresholds.head()

,Pollutant,Min Value,Threshold Adhered To
2,aldicarb,35.0,Prop65_Cancer
3,copper,6300.0,USEPA_IRIS_RfD
5,boron,700.0,USEPA_IRIS_RfD
6,terbacil,1.0,USEPA_HA_Cancer
8,bromoxynil,0.4,USEPA_MCL_Goal


In [100]:
common_pollutants_p = []
reverse_params_def_dict = {}
for pollutant in ca_pollutant_thresholds["Pollutant"]:
    for column in all_filtered_params[pollutant]:
        reverse_params_def_dict[column] = pollutant
        common_pollutants_p.append(column)

In [101]:
ca_pollutant_thresholds.shape

(129, 3)

In [102]:
common_pollutants_p

170

In [103]:
# Parallel test for new pollutant data - WORKS
column = "p01010"
threshold = ca_pollutant_thresholds["Min Value"][ca_pollutant_thresholds["Pollutant"]==reverse_params_def_dict[column]]
actual_threshold = threshold.values[0]

In [104]:
# passed_threshold = gives gauges that exceeded threshold
passed_threshold = total_data[column][total_data[column].notna()] > actual_threshold
has_passed_site = total_data["site_no"][total_data[column].notna()][passed_threshold]
has_passed_nums = total_data[column][total_data[column].notna()][passed_threshold]

In [105]:
total_data.columns.values.tolist()

['site_no',
 'sample_dt',
 'sample_tm',
 'p61591',
 'p49312',
 'p01040',
 'p01041',
 'p01043',
 'p01020',
 'p04032',
 'p82665',
 'p49311',
 'p49299',
 'p00950',
 'p91002',
 'p09503',
 'p09511',
 'p49296',
 'p39333',
 'p63188',
 'p01044',
 'p01046',
 'p01170',
 'p39415',
 'p63218',
 'p04025',
 'p82673',
 'p00945',
 'p82677',
 'p38454',
 'p71865',
 'p39632',
 'p63182',
 'p63194',
 'p82664',
 'p04041',
 'p63183',
 'p39363',
 'p49292',
 'p01054',
 'p01056',
 'p63168',
 'p49301',
 'p01065',
 'p01066',
 'p01068',
 'p49315',
 'p01030',
 'p01031',
 'p01035',
 'p01036',
 'p63202',
 'p04028',
 'p68824',
 'p68832',
 'p81366',
 'p38478',
 'p82666',
 'p01060',
 'p38711',
 'p39341',
 'p39343',
 'p04037',
 'p63226',
 'p63222',
 'p38811',
 'p00721',
 'p49313',
 'p04029',
 'p63189',
 'p68826',
 'p68834',
 'p01057',
 'p01145',
 'p01146',
 'p01148',
 'p04035',
 'p39393',
 'p39373',
 'p50471',
 'p63187',
 'p63227',
 'p00618',
 'p71851',
 'p39403',
 'p46342',
 'p82681',
 'p63163',
 'p63225',
 'p00940',
 'p

In [92]:
to_write_to_file = ""
header = "Site\tPollutant\tDate\t% Error\tValue\tThreshold\tThreshold Adhered To\n"
formatSTR = "{}\t{}\t{}\t{}\t{}\t{}\t{}\n"
pollutant_col = "Pollutant"
threshold_col = "Min Value"
which_threshold_col = "Threshold Adhered To"
date_col = "sample_dt"

In [93]:
reverse_params_def_dict

{'p49312': 'aldicarb',
 'p01040': 'copper',
 'p01041': 'copper',
 'p01043': 'copper',
 'p01020': 'boron',
 'p04032': 'terbacil',
 'p82665': 'terbacil',
 'p49311': 'bromoxynil',
 'p49299': '2-methyl-4,6-dinitrophenol',
 'p00950': 'fluoride',
 'p91002': 'fluoride',
 'p09503': 'radium-226',
 'p09511': 'radium-226',
 'p49296': 'methomyl',
 'p39333': 'aldrin',
 'p63188': 'bisphenol a',
 'p01044': 'iron',
 'p01046': 'iron',
 'p01170': 'iron',
 'p39415': 'metolachlor',
 'p63218': 'metolachlor',
 'p04025': 'hexazinone',
 'p82673': 'benfluralin',
 'p00945': 'sulfate',
 'p82677': 'disulfoton',
 'p38454': 'dicrotophos',
 'p71865': 'iodide',
 'p63194': 'carbazole',
 'p82664': 'phorate',
 'p04041': 'cyanazine',
 'p63183': 'benzo[a]pyrene',
 'p39363': "p,p'-ddd",
 'p49292': 'oryzalin',
 'p01054': 'manganese',
 'p01056': 'manganese',
 'p49301': 'dinoseb',
 'p01065': 'nickel',
 'p01066': 'nickel',
 'p01068': 'nickel',
 'p49315': 'acifluorfen',
 'p01030': 'chromium',
 'p01031': 'chromium',
 'p01035': '

In [106]:
# Going through all the p##### column names
#  use the pollutant associated to that column to determine where there are rows for that
#  pollutant
# The 
for column in common_pollutants_p:
    columns_with_pollutant_info = ca_pollutant_thresholds[pollutant_col]==reverse_params_def_dict[column]
    threshold = ca_pollutant_thresholds[threshold_col][columns_with_pollutant_info].values[0]
    passed_threshold = total_data[column][total_data[column].notna()] > threshold
    has_passed_site = total_data["site_no"][total_data[column].notna()][passed_threshold]
    has_passed_nums = total_data[column][total_data[column].notna()][passed_threshold]
    has_passed_pcts = (has_passed_nums - threshold)/threshold
    has_passed_date = total_data[date_col][total_data[column].notna()][passed_threshold]
    which_threshold = ca_pollutant_thresholds[which_threshold_col][ca_pollutant_thresholds[pollutant_col]==reverse_params_def_dict[column]].values[0]
    for i in range(len(has_passed_site)):
        to_add = formatSTR.format(has_passed_site.iloc[i], reverse_params_def_dict[column], 
                                  has_passed_date.iloc[i], has_passed_pcts.iloc[i], 
                                  has_passed_nums.iloc[i], threshold, which_threshold)
        to_write_to_file += to_add

In [108]:
file = open("bipartite_CA.tsv", "w")
file.write(header + to_write_to_file)
file.close()